<a href="https://colab.research.google.com/github/Gus-1003/ISD-invention/blob/main/Eletro_1/Coordinates_of_an_object_in_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas:

## Manipulação de Dados:

In [1]:
# Importa a biblioteca glob para obter uma lista de caminhos de arquivos que correspondem a um determinado padrão.
import glob
# Importa a biblioteca OpenCV para processamento de imagem.
import cv2
# Importa a biblioteca NumPy para operações numéricas.
import numpy as np

import pandas as pd

## OpenCV

In [2]:
from google.colab.patches import cv2_imshow

## Drive

In [3]:
from google.colab.patches import cv2_imshow

from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/

Mounted at /content/drive/
/content/drive/MyDrive


# Leitura do Pacote de Imagens

In [4]:
# Variável path_list armazena uma lista de caminhos de arquivos que correspondem ao padrão especificado.

path_list = glob.glob('/content/drive/MyDrive/Pesquisa_ISD_IC/Base de Dados/*.jpg')

#path_list = glob.glob('/content/drive/MyDrive/Pesquisa_ISD_IC/Images_teste/*.jpg')

# Algoritmo de identificação:

## Criação de Series para armazenar dados:

In [5]:
nomes = []
formas = []
areas = []
circularitys = []
radios = []

## Processamento:

Estudo do OpenCV: https://learnopencv.com/getting-started-with-opencv/

In [ ]:
# Loop que processa cada arquivo na lista de caminhos de arquivos.
for frame in path_list:

  # The function cv2.imread() is used to read an image.
  im = cv2.imread(frame)

  # Separa o nome especifico da imagem.
  nome_completo = frame.split('/')
  nome = nome_completo[6].split('.')
  nome[0]

  # Imprime as fronteiras e identifica as imagens:
  print("================================================================================")
  print("================================================================================") 
  print(nome[0])

  # Cria uma cópia da imagem original.
  img_contours = im.copy()

  # Converte a imagem para o espaço de cores em escala de cinza.
  image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  
  # Converte a imagem para o tipo float32.
  image32f = np.float32(image)

  # Aplica um filtro de média à imagem.
  mu = cv2.blur(image32f, (3, 3))

  # Aplica um filtro de variância à imagem.
  mu2 = cv2.blur(image32f*image32f, (3, 3))
  sigma = cv2.sqrt(mu2 - mu*mu)
  sigma = sigma * 10
  sigma = sigma.astype("uint8")

  # Aplica um filtro de borramento com kernel de 5x5
  sigma = cv2.GaussianBlur(sigma, (5, 5), 0)  

  # Aplica uma limiarização com método de Otsu à imagem.
  ret3,th3 = cv2.threshold(sigma,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

  # Define o kernel para dilatação e erosão
  kernel = np.ones((3,3), np.uint8)

  # Dilata a imagem
  img_dilated = cv2.dilate(th3, kernel, iterations=2)

  # Erode a imagem
  th3 = cv2.erode(img_dilated, kernel, iterations=2)

  # Encontra os contornos na imagem limiarizada.
  contours, hierarchy = cv2.findContours(th3, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # https://doi.org/10.1016/0734-189X(85)90016-7

  # Loop sobre os contornos
  for cnt in contours:

    # Aproxima o contorno por uma sequência de pontos
    approx = cv2.approxPolyDP(cnt, 0.01 * cv2.arcLength(cnt, True), True)

    # Calcula a área e o perímetro do contorno
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)

    # Verifica se o contorno é um círculo
    if perimeter > 0:
      circularity = 4 * np.pi * area / (perimeter ** 2)
    else:
      circularity = 0

    # Desenha um círculo
    if circularity > 0.39:

      forma = "circulo"

      (x, y), radius = cv2.minEnclosingCircle(cnt)

      # Limita o raio máximo do círculo
      max_radius = 100  # Define o raio máximo desejado
      radius = min(radius, max_radius)  # Limita o raio ao valor máximo

      center = (int(x), int(y))
      radius = int(radius)

      #if area > 2000 or circularity > 53:
      if area > 2000:
        if radius < 100 and radius > 20:
          cv2.circle(img_contours, center, radius, (255, 0, 0), 2)
      
      print("circularity (Circulo): " + str(circularity)  + " / Area (Circulo): " + str(area)  + " / radius (Circulo): " + str(radius))

    # Desenha um quadrado
    else:
      rect = cv2.minAreaRect(cnt)
      
      forma = "quadrado"
      radius = 0

      (x, y), (w, h), angle = rect

      if min(w, h) == 0:
        aspect_ratio = 0
      else:
        aspect_ratio = max(w, h) / min(w, h)
      
      print("circularity (Quadrado): " + str(circularity)  + " / Area (Quadrado): " + str(area)  + " / aspect_ratio (Quadrado): " + str(aspect_ratio))

      #if aspect_ratio < 1.1:
      if aspect_ratio < 3:  
        # Desenha um quadrado
        box = cv2.boxPoints(rect)
        box = np.int0(box)

        if area > 65:
          cv2.drawContours(img_contours, [box], 0, (0, 0, 255), 2)
      


    nomes.append(nome[0])
    formas.append(forma)
    circularitys.append(circularity)
    areas.append(area)
    radios.append(radius)

  # Exibe a imagem filtro de média
  #cv2_imshow(mu)
  # Exibe a imagem pos-filtro.
  cv2_imshow(sigma)
  # Exibe a imagem limiarizada.
  cv2_imshow(th3)
  # Exibe a imagem original copiada com os contornos encontrados.
  cv2_imshow(img_contours)
  print()
  print()
  print()

# Construção de um dataset:

In [7]:
features_1 = {'nome': nomes, 'forma': formas, 'area': areas, 'cicularity': circularitys, 'aspect_ratio': aspect_ratio, 'radius': radios}

In [8]:
count = 0

nome_completo = frame.split('/')
nome = nome_completo[6].split('.')
nome[0]

'Somatoria35'

In [ ]:
df = pd.DataFrame(features_1) 
df